In [5]:
import pandas as pd
import os
print(os.getcwd())


df = pd.read_csv('../data/raw/uber.csv') 

print(df.head())
# IF FIRST FEW LINES OF CSV LOAD EVERYTHINGS GOOD :)

/Users/mattrodosky/Desktop/repos/Project-2/notebooks
   Unnamed: 0                            key  fare_amount  \
0    24238194    2015-05-07 19:52:06.0000003          7.5   
1    27835199    2009-07-17 20:04:56.0000002          7.7   
2    44984355   2009-08-24 21:45:00.00000061         12.9   
3    25894730    2009-06-26 08:22:21.0000001          5.3   
4    17610152  2014-08-28 17:47:00.000000188         16.0   

           pickup_datetime  pickup_longitude  pickup_latitude  \
0  2015-05-07 19:52:06 UTC        -73.999817        40.738354   
1  2009-07-17 20:04:56 UTC        -73.994355        40.728225   
2  2009-08-24 21:45:00 UTC        -74.005043        40.740770   
3  2009-06-26 08:22:21 UTC        -73.976124        40.790844   
4  2014-08-28 17:47:00 UTC        -73.925023        40.744085   

   dropoff_longitude  dropoff_latitude  passenger_count  
0         -73.999512         40.723217                1  
1         -73.994710         40.750325                1  
2         -73.9

In [6]:
def clean_uber_data(data):
    """
    Clean the Uber trip data.

    Parameters:
        data (DataFrame): The Uber trip data to clean.

    Returns:
        DataFrame: The cleaned Uber trip data.
    """

    # dont touch pls :) converts to normal datetime 
    if 'pickup_datetime' in data.columns:
        data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'], errors='coerce')
        data['pickup_time'] = data['pickup_datetime'].dt.time  # New column with only time
 
    data.drop_duplicates(inplace=True)

    # MISSING VALUES DROPPER
    data.dropna(subset=['key', 'fare_amount'], inplace=True)

    # When passenger amount missing it fills it with a big fat 0
    if 'passenger_count' in data.columns:
        data['passenger_count'].fillna(0, inplace=True)

    # Sometimes data imported from CSV files may contain mixed types 
    # (some entries may be strings, others could be numbers, etc.). 
    # This can happen due to formatting issues in the CSV file.

    # I did research on .astype(float) because I didnt fully understand so my basic findings are above
    if 'fare_amount' in data.columns:
        data['fare_amount'] = data['fare_amount'].astype(float)

    return data

In [7]:
cleaned_uber_data = clean_uber_data(df)

/var/folders/fx/7k4d70096813w2_tf7kjpzgr0000gn/T/ipykernel_49417/2860180699.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['passenger_count'].fillna(0, inplace=True)


In [10]:
cleaned_data_path = '../data/processed/uber_data_cleaned.csv' 
cleaned_uber_data.to_csv(cleaned_data_path, index=False)

print(f"Cleaned Uber data saved to {cleaned_data_path}.")
print(cleaned_uber_data.info())
# IF LOADS MESSAGE ABOVE BE HAPPY 
#     ∧＿∧
# 　 (｡･ω･｡)つ━☆・*。
#  ⊂/　  /　   ・゜
# 　しーＪ　　　  °。+ * 。　
# 　　　　　         .・゜
# 　　　　　          ゜｡ﾟﾟ･｡･ﾟﾟ。
# 　　　　           　ﾟ。    ｡ﾟ
#                     　ﾟ･｡･ﾟ       


Cleaned Uber data saved to ../data/processed/uber_data_cleaned.csv.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   Unnamed: 0         200000 non-null  int64              
 1   key                200000 non-null  object             
 2   fare_amount        200000 non-null  float64            
 3   pickup_datetime    200000 non-null  datetime64[ns, UTC]
 4   pickup_longitude   200000 non-null  float64            
 5   pickup_latitude    200000 non-null  float64            
 6   dropoff_longitude  199999 non-null  float64            
 7   dropoff_latitude   199999 non-null  float64            
 8   passenger_count    200000 non-null  int64              
 9   pickup_time        200000 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(5), int64(2), object(2)
memory usage: 15.3+ MB
No